In [2]:
import os
from dotenv import load_dotenv
import mysql.connector
import requests
import pandas as pd
from datetime import datetime, timedelta
from IPython.display import HTML, display
from openpyxl import Workbook
from openpyxl.drawing.image import Image
from mysql.connector import Error

In [1]:
# Function to connect to MySQL database
def connect_to_mysql():
    try:
        connection = mysql.connector.connect(
            host=MYSQL_HOST,  # Use variables loaded from dotenv
            database=MYSQL_DATABASE,
            user=MYSQL_USER,
            password=MYSQL_PASSWORD,
        )
        if connection.is_connected():
            print("Connected to MySQL database")
            return connection
    except Error as e:
        print(f"Error: {e}")
        return None

# Function to insert data into MySQL
def insert_data_to_mysql(df, connection):
    try:
        if connection.is_connected():
            cursor = connection.cursor()
            insert_query = """
                INSERT INTO weather_data (
                    time, temperature, temperature_apparent, humidity, rain, wind_speed, weather_code, weather_icon
                )
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            """
            # Insert each row from the DataFrame
            for row in df.itertuples(index=False):
                cursor.execute(insert_query, (
                    row.time, 
                    row.temperature, 
                    row.temperatureApparent, 
                    row.humidity, 
                    row.rain, 
                    row.windSpeed, 
                    row.weather_code, 
                    row.weather_icon
                ))
            connection.commit()
            print(f"Inserted {cursor.rowcount} rows successfully")
    except Error as e:
        print(f"Error during data insertion: {e}")
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")

# Ensure connection is established
mysql_connection = connect_to_mysql()

if mysql_connection:
    insert_data_to_mysql(weather_df, mysql_connection)
else:
    print("Failed to connect to MySQL database.")

NameError: name 'Error' is not defined